In [1]:
import pyspark as spark
import numpy as np
import pandas as pd
import sklearn
import geopandas as gpd 
from geopandas import GeoDataFrame
from shapely.geometry import Point, Polygon
import matplotlib.pyplot as plt


In [2]:
crash = pd.read_csv('../data/Traffic_Crashes_-_Crashes.csv')

In [3]:
vehicles = pd.read_csv('../data/Traffic_Crashes_-_Vehicles.csv')

/Users/winsometang/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (19,21,40,41,42,44,48,49,50,53,55,58,59,61,71) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
people = pd.read_csv('../data/Traffic_Crashes_-_People.csv')

/Users/winsometang/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (12,13,20,21,22,23,24,25,26,27,29) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
crash.describe()

,POSTED_SPEED_LIMIT,LANE_CNT,STREET_NO,BEAT_OF_OCCURRENCE,NUM_UNITS,INJURIES_TOTAL,INJURIES_FATAL,INJURIES_INCAPACITATING,INJURIES_NON_INCAPACITATING,INJURIES_REPORTED_NOT_EVIDENT,INJURIES_NO_INDICATION,INJURIES_UNKNOWN,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,LATITUDE,LONGITUDE
count,517282.000000,1.989660e+05,517282.000000,517277.000000,517282.000000,516244.00000,516244.000000,516244.000000,516244.000000,516244.000000,516244.000000,516244.0,517282.000000,517282.000000,517282.000000,514382.000000,514382.000000
mean,28.302120,1.333242e+01,3661.327804,1239.329638,2.033218,0.17882,0.001060,0.019473,0.099629,0.058658,2.021995,0.0,13.242158,4.126892,6.573674,41.855046,-87.673046
std,6.444674,2.961921e+03,2910.696396,707.065267,0.445490,0.55063,0.035331,0.163458,0.408358,0.311175,1.169575,0.0,5.506523,1.977442,3.438811,0.336088,0.683169
min,0.000000,0.000000e+00,0.000000,111.000000,1.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,1.000000,1.000000,0.000000,-87.934763
25%,30.000000,2.000000e+00,1215.000000,712.000000,2.000000,0.00000,0.000000,0.000000,0.000000,0.000000,1.000000,0.0,9.000000,2.000000,4.000000,41.782219,-87.721285
50%,30.000000,2.000000e+00,3198.000000,1211.000000,2.000000,0.00000,0.000000,0.000000,0.000000,0.000000,2.000000,0.0,14.000000,4.000000,7.000000,41.875409,-87.673455
75%,30.000000,4.000000e+00,5582.000000,1822.000000,2.000000,0.00000,0.000000,0.000000,0.000000,0.000000,2.000000,0.0,17.000000,6.000000,10.000000,41.924046,-87.632828
max,99.000000,1.191625e+06,451100.000000,6100.000000,18.000000,21.00000,4.000000,7.000000,21.000000,15.000000,61.000000,0.0,23.000000,7.000000,12.000000,42.022780,0.000000


In [6]:
people.describe()

,VEHICLE_ID,SEAT_NO,AGE,BAC_RESULT VALUE
count,1.120504e+06,233668.000000,815745.000000,1460.000000
mean,5.337715e+05,4.165466,37.847617,0.167473
std,3.072029e+05,2.186325,17.106627,0.099561
min,2.000000e+00,1.000000,-177.000000,0.000000
25%,2.690950e+05,3.000000,25.000000,0.120000
50%,5.330980e+05,3.000000,35.000000,0.170000
75%,7.969340e+05,6.000000,50.000000,0.220000
max,1.071291e+06,12.000000,110.000000,1.000000


In [7]:
vehicles.describe()

,CRASH_UNIT_ID,UNIT_NO,NUM_PASSENGERS,VEHICLE_ID,VEHICLE_YEAR,OCCUPANT_CNT,CMV_ID,TRAILER1_LENGTH,TRAILER2_LENGTH,TOTAL_VEHICLE_LENGTH,AXLE_CNT
count,1.057474e+06,1.057474e+06,158436.000000,1.033331e+06,865317.000000,1.033331e+06,11123.000000,1847.000000,49.000000,2222.000000,3219.000000
mean,5.642652e+05,5.134954e+00,1.474842,5.373088e+05,2013.451148,1.083948e+00,6198.145644,47.964266,45.204082,52.823582,11.803976
std,3.276157e+05,3.673931e+03,1.072623,3.094255e+05,152.780094,7.931411e-01,3599.640927,15.593072,30.024975,33.124591,458.362863
min,2.000000e+00,0.000000e+00,1.000000,2.000000e+00,1900.000000,0.000000e+00,1.000000,1.000000,2.000000,1.000000,1.000000
25%,2.790312e+05,1.000000e+00,1.000000,2.696445e+05,2006.000000,1.000000e+00,3079.500000,42.500000,24.000000,35.000000,2.000000
50%,5.645995e+05,2.000000e+00,1.000000,5.378600e+05,2012.000000,1.000000e+00,6125.000000,53.000000,48.000000,53.000000,3.000000
75%,8.490518e+05,2.000000e+00,2.000000,8.059005e+05,2015.000000,1.000000e+00,9325.500000,53.000000,53.000000,65.000000,5.000000
max,1.129364e+06,3.778035e+06,59.000000,1.071292e+06,9999.000000,9.900000e+01,12434.000000,200.000000,123.000000,999.000000,26009.000000


In [8]:
people.head(10)

,PERSON_ID,PERSON_TYPE,CRASH_RECORD_ID,RD_NO,VEHICLE_ID,CRASH_DATE,SEAT_NO,CITY,STATE,ZIPCODE,...,EMS_RUN_NO,DRIVER_ACTION,DRIVER_VISION,PHYSICAL_CONDITION,PEDPEDAL_ACTION,PEDPEDAL_VISIBILITY,PEDPEDAL_LOCATION,BAC_RESULT,BAC_RESULT VALUE,CELL_PHONE_USE
0,O749947,DRIVER,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,JC451435,834816.0,09/28/2019 03:30:00 AM,NaN,CHICAGO,IL,60651,...,NaN,UNKNOWN,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
1,O871921,DRIVER,af84fb5c8d996fcd3aefd36593c3a02e6e7509eeb27568...,JD208731,827212.0,04/13/2020 10:50:00 PM,NaN,CHICAGO,IL,60620,...,NaN,NONE,NOT OBSCURED,NORMAL,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
2,O10018,DRIVER,71162af7bf22799b776547132ebf134b5b438dcf3dac6b...,HY484534,9579.0,11/01/2015 05:00:00 AM,NaN,NaN,NaN,NaN,...,NaN,IMPROPER BACKING,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
3,O10038,DRIVER,c21c476e2ccc41af550b5d858d22aaac4ffc88745a1700...,HY484750,9598.0,11/01/2015 08:00:00 AM,NaN,NaN,NaN,NaN,...,NaN,UNKNOWN,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
4,O10039,DRIVER,eb390a4c8e114c69488f5fb8a097fe629f5a92fd528cf4...,HY484778,9600.0,11/01/2015 10:15:00 AM,NaN,NaN,NaN,NaN,...,NaN,UNKNOWN,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
5,O10041,DRIVER,dd1bce4bd6d0be4c247714dcabab44e6563c62b913229b...,HY484775,9601.0,11/01/2015 11:00:00 AM,NaN,NaN,NaN,NaN,...,NaN,UNKNOWN,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
6,O10062,DRIVER,4bd2ee6bb306902b99a9c2ae55cf4fcffec00879e39759...,HY484984,9621.0,11/01/2015 12:30:00 PM,NaN,NaN,NaN,NaN,...,NaN,UNKNOWN,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
7,O10066,DRIVER,9c03b6fcc6d72cf3ee2cb9ea754ea7e4617ab965142552...,HY484988,9623.0,10/31/2015 04:00:00 PM,NaN,NaN,NaN,NaN,...,NaN,UNKNOWN,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
8,O1007,DRIVER,2e7e0c1682100a200a46c25a532eda4febb00a2ae49ea9...,HY407490,958.0,09/02/2015 02:35:00 PM,NaN,NaN,NaN,NaN,...,NaN,UNKNOWN,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
9,O10088,DRIVER,8bf8069b3d839b732fd35d2c9d8caddeddbb4b8978e84d...,HY485119,9645.0,11/01/2015 01:00:00 PM,NaN,NaN,NaN,NaN,...,NaN,UNKNOWN,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN


In [9]:
crash.head(10)

,CRASH_RECORD_ID,RD_NO,CRASH_DATE_EST_I,CRASH_DATE,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,DEVICE_CONDITION,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,...,INJURIES_NON_INCAPACITATING,INJURIES_REPORTED_NOT_EVIDENT,INJURIES_NO_INDICATION,INJURIES_UNKNOWN,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,LATITUDE,LONGITUDE,LOCATION
0,4fd0a3e0897b3335b94cd8d5b2d2b350eb691add56c62d...,JC343143,NaN,07/10/2019 05:56:00 PM,35,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,TURNING,...,0.0,0.0,3.0,0.0,17,4,7,41.919664,-87.773288,POINT (-87.773287883007 41.919663832993)
1,009e9e67203442370272e1a13d6ee51a4155dac65e583d...,JA329216,NaN,06/30/2017 04:00:00 PM,35,STOP SIGN/FLASHER,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,TURNING,...,0.0,0.0,3.0,0.0,16,6,6,41.741804,-87.740954,POINT (-87.740953581987 41.741803598989)
2,ee9283eff3a55ac50ee58f3d9528ce1d689b1c4180b4c4...,JD292400,NaN,07/10/2020 10:25:00 AM,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,REAR END,...,0.0,0.0,3.0,0.0,10,6,7,41.773456,-87.585022,POINT (-87.585022352022 41.773455972008)
3,f8960f698e870ebdc60b521b2a141a5395556bc3704191...,JD293602,NaN,07/11/2020 01:00:00 AM,30,NO CONTROLS,NO CONTROLS,CLEAR,DARKNESS,PARKED MOTOR VEHICLE,...,0.0,0.0,3.0,0.0,1,7,7,41.802119,-87.622115,POINT (-87.622114914961 41.802118543011)
4,8eaa2678d1a127804ee9b8c35ddf7d63d913c14eda61d6...,JD290451,NaN,07/08/2020 02:00:00 PM,20,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,PARKED MOTOR VEHICLE,...,0.0,0.0,1.0,0.0,14,4,7,NaN,NaN,NaN
5,00e47f189660cd8ba1e85fc63061bf1d8465184393f134...,JC194776,NaN,03/21/2019 10:50:00 PM,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",TURNING,...,0.0,0.0,2.0,0.0,22,5,3,41.741804,-87.740954,POINT (-87.740953581987 41.741803598989)
6,0126747fc9ffc0edc9a38abb83d80034f897db0f739eef...,JB200478,NaN,03/26/2018 02:23:00 PM,35,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,PARKED MOTOR VEHICLE,...,0.0,0.0,2.0,0.0,14,2,3,41.953647,-87.732082,POINT (-87.732081736006 41.953646899951)
7,f636d4a51a88015ac89031159b1f1952b8d92e49d11aeb...,JD293237,NaN,07/10/2020 10:20:00 PM,30,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",SIDESWIPE OPPOSITE DIRECTION,...,1.0,0.0,1.0,0.0,22,6,7,41.880386,-87.725807,POINT (-87.725807044026 41.880385884998)
8,76aabcf7c2219a5c90259c96fe94b33834ddb53e0dbcd3...,JD291826,NaN,07/09/2020 05:06:00 PM,10,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,PARKED MOTOR VEHICLE,...,0.0,0.0,1.0,0.0,17,5,7,NaN,NaN,NaN
9,9c974548026c1b962569040bd8fa08ae643ffc28c15ebd...,JD280446,NaN,06/29/2020 05:55:00 PM,10,OTHER,NO CONTROLS,CLOUDY/OVERCAST,DAWN,PARKED MOTOR VEHICLE,...,0.0,0.0,1.0,0.0,17,2,6,NaN,NaN,NaN


In [10]:
vehicles.head(10)

,CRASH_UNIT_ID,CRASH_RECORD_ID,RD_NO,CRASH_DATE,UNIT_NO,UNIT_TYPE,NUM_PASSENGERS,VEHICLE_ID,CMRC_VEH_I,MAKE,...,TRAILER1_LENGTH,TRAILER2_LENGTH,TOTAL_VEHICLE_LENGTH,AXLE_CNT,VEHICLE_CONFIG,CARGO_BODY_TYPE,LOAD_TYPE,HAZMAT_OUT_OF_SERVICE_I,MCS_OUT_OF_SERVICE_I,HAZMAT_CLASS
0,829999,24ddf9fd8542199d832e1c223cc474e5601b356f1d77a6...,JD124535,01/22/2020 06:25:00 AM,1,DRIVER,NaN,796949.0,NaN,INFINITI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,749947,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,JC451435,09/28/2019 03:30:00 AM,1,DRIVER,NaN,834816.0,NaN,HONDA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,749949,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,JC451435,09/28/2019 03:30:00 AM,2,PARKED,NaN,834819.0,NaN,TOYOTA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,749950,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,JC451435,09/28/2019 03:30:00 AM,3,PARKED,NaN,834817.0,NaN,GENERAL MOTORS CORPORATION (GMC),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,871921,af84fb5c8d996fcd3aefd36593c3a02e6e7509eeb27568...,JD208731,04/13/2020 10:50:00 PM,2,DRIVER,NaN,827212.0,NaN,BUICK,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,473653,80f5b6eff349a72094c9c7a11745549caa8bd39b57cedd...,JB428391,09/09/2018 08:50:00 AM,2,PARKED,NaN,453462.0,NaN,LINCOLN-CONTINENTAL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,834812,ecf9e646f89a73ed3df0f373e4fbd18e91f5a595004019...,JD138756,02/03/2020 03:45:00 PM,1,DRIVER,NaN,792247.0,NaN,HONDA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,834813,ecf9e646f89a73ed3df0f373e4fbd18e91f5a595004019...,JD138756,02/03/2020 03:45:00 PM,2,DRIVER,1.0,792252.0,NaN,SUBARU,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,837380,c7c57a49274b3aa953d0b76346f68e116ed8c3d72a683a...,JD143282,02/06/2020 12:00:00 AM,1,DRIVER,NaN,794648.0,NaN,UNKNOWN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,837381,c7c57a49274b3aa953d0b76346f68e116ed8c3d72a683a...,JD143282,02/06/2020 12:00:00 AM,2,PARKED,NaN,794653.0,NaN,INFINITI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
vehicles.isnull().sum().head(19)

CRASH_UNIT_ID             0
CRASH_RECORD_ID           0
RD_NO                  9722
CRASH_DATE                0
UNIT_NO                   0
UNIT_TYPE              1552
NUM_PASSENGERS       899038
VEHICLE_ID            24143
CMRC_VEH_I          1037860
MAKE                  24148
MODEL                 24291
LIC_PLATE_STATE      113487
VEHICLE_YEAR         192157
VEHICLE_DEFECT        24143
VEHICLE_TYPE          24143
VEHICLE_USE           24143
TRAVEL_DIRECTION      24143
MANEUVER              24143
TOWED_I              935389
dtype: int64

In [12]:
people['INJURY_CLASSIFICATION'].value_counts()

NO INDICATION OF INJURY     1048927
NONINCAPACITATING INJURY      52067
REPORTED, NOT EVIDENT         30568
INCAPACITATING INJURY         10211
FATAL                           606
Name: INJURY_CLASSIFICATION, dtype: int64

In [13]:
people['INJURY_CLASSIFICATION'].isnull().value_counts()

False    1142379
True         584
Name: INJURY_CLASSIFICATION, dtype: int64

In [14]:
people=people[people['INJURY_CLASSIFICATION'].isnull()==False]

In [15]:
people.shape

(1142379, 30)

In [16]:
severe_injury =people[people['INJURY_CLASSIFICATION'].isin(['FATAL','INCAPACITATING INJURY'])]

In [17]:
severe_injury.columns

Index(['PERSON_ID', 'PERSON_TYPE', 'CRASH_RECORD_ID', 'RD_NO', 'VEHICLE_ID',
       'CRASH_DATE', 'SEAT_NO', 'CITY', 'STATE', 'ZIPCODE', 'SEX', 'AGE',
       'DRIVERS_LICENSE_STATE', 'DRIVERS_LICENSE_CLASS', 'SAFETY_EQUIPMENT',
       'AIRBAG_DEPLOYED', 'EJECTION', 'INJURY_CLASSIFICATION', 'HOSPITAL',
       'EMS_AGENCY', 'EMS_RUN_NO', 'DRIVER_ACTION', 'DRIVER_VISION',
       'PHYSICAL_CONDITION', 'PEDPEDAL_ACTION', 'PEDPEDAL_VISIBILITY',
       'PEDPEDAL_LOCATION', 'BAC_RESULT', 'BAC_RESULT VALUE',
       'CELL_PHONE_USE'],
      dtype='object')

In [18]:
severe_injury[severe_injury['INJURY_CLASSIFICATION']== 'FATAL']

,PERSON_ID,PERSON_TYPE,CRASH_RECORD_ID,RD_NO,VEHICLE_ID,CRASH_DATE,SEAT_NO,CITY,STATE,ZIPCODE,...,EMS_RUN_NO,DRIVER_ACTION,DRIVER_VISION,PHYSICAL_CONDITION,PEDPEDAL_ACTION,PEDPEDAL_VISIBILITY,PEDPEDAL_LOCATION,BAC_RESULT,BAC_RESULT VALUE,CELL_PHONE_USE
504,O838563,DRIVER,362c12430744ace0e1bd90e95689790166481ef62c096f...,JD145123,795783.0,02/08/2020 09:47:00 PM,NaN,CHICAGO,IL,60639,...,NaN,NONE,UNKNOWN,NORMAL,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
5130,O876584,PEDESTRIAN,156c7876ed4bc4a5885189778b871d0d304477d379a815...,JD217176,NaN,04/25/2020 08:34:00 PM,NaN,UNKNOWN,NaN,NaN,...,NaN,NaN,NaN,NaN,CROSSING - NO CONTROLS (AT INTERSECTION),NO CONTRASTING CLOTHING,IN CROSSWALK,TEST NOT OFFERED,NaN,NaN
6599,O852152,BICYCLE,fdc5d4d1299102e75fd8c7997ea5c343a0b5e62a4f7bde...,JD168667,NaN,02/29/2020 12:35:00 AM,NaN,NaN,NaN,NaN,...,NaN,UNKNOWN,UNKNOWN,REMOVED BY EMS,CROSSING - NO CONTROLS (NOT AT INTERSECTION),NO CONTRASTING CLOTHING,IN ROADWAY,NaN,NaN,NaN
8830,O882908,DRIVER,80b1325c9066353b979677290e930e876ceadb355a6469...,JD230760,837598.0,05/12/2020 07:00:00 PM,NaN,FLOSSMOOR,IL,60422,...,NaN,UNKNOWN,UNKNOWN,REMOVED BY EMS,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
10062,O853602,DRIVER,08d8b1747ca790951f856dcd6bf1e45aebf16cc6a0b5d1...,JD170887,810042.0,03/02/2020 02:07:00 AM,NaN,TINLEY PARK,IL,60477,...,NaN,TOO FAST FOR CONDITIONS,NOT OBSCURED,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1131904,P252255,PASSENGER,a7b66672d32baa415862a2c1b777d5eb4f4ea7dd596631...,NaN,1065852.0,06/21/2021 09:44:00 PM,3.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1136348,O1125801,DRIVER,cecfca2ab8941e918a9221cb1bcf414a7f8c23abdfa109...,NaN,1067876.0,06/24/2021 11:15:00 PM,NaN,CHICAGO,IL,60653,...,NaN,OTHER,UNKNOWN,REMOVED BY EMS,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
1137626,O1126307,DRIVER,98f24fedc8914d65e3fd08b1d3e6c1ab731b163bf5d19b...,NaN,1068359.0,06/25/2021 02:27:00 PM,NaN,CHICAGO,IL,60619,...,NaN,OTHER,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
1139332,O1127533,DRIVER,f1f084a5a927eb66b19e82b2460b328d33de503a70428a...,NaN,1069545.0,06/27/2021 10:54:00 AM,NaN,BERWYN,IL,60402,...,NaN,UNKNOWN,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN


In [19]:
people=people[people['INJURY_CLASSIFICATION'].isnull()==False]

In [20]:
clean_people=people[['CRASH_RECORD_ID',"VEHICLE_ID",'CRASH_DATE','INJURY_CLASSIFICATION',"AIRBAG_DEPLOYED",'DRIVER_ACTION','PEDPEDAL_ACTION','BAC_RESULT VALUE','CELL_PHONE_USE']]

In [21]:
clean_people.shape

(1142379, 9)

In [22]:
clean_vehicles = vehicles[['CRASH_UNIT_ID',"CRASH_RECORD_ID","CRASH_DATE",'MAKE','MODEL','VEHICLE_TYPE','VEHICLE_DEFECT','FIRST_CONTACT_POINT','FIRE_I']]

In [23]:
clean_vehicles.shape

(1057474, 9)

In [24]:
crash_clean =crash[['CRASH_RECORD_ID','CRASH_DATE','WEATHER_CONDITION','LIGHTING_CONDITION','LANE_CNT','ROADWAY_SURFACE_COND','ROAD_DEFECT','TRAFFICWAY_TYPE','STREET_NO','STREET_NAME','BEAT_OF_OCCURRENCE','WORK_ZONE_TYPE','INJURIES_TOTAL','MOST_SEVERE_INJURY','INJURIES_FATAL','CRASH_MONTH','CRASH_DAY_OF_WEEK','TRAFFICWAY_TYPE','CRASH_HOUR','LATITUDE','LONGITUDE','LOCATION']]

In [25]:
crash_clean.shape

(517282, 22)

In [26]:
mergeddf=clean_people.merge(clean_vehicles,on='CRASH_RECORD_ID').merge(crash_clean,on='CRASH_RECORD_ID')

In [27]:
mergeddf.describe()

,VEHICLE_ID,BAC_RESULT VALUE,CRASH_UNIT_ID,LANE_CNT,STREET_NO,BEAT_OF_OCCURRENCE,INJURIES_TOTAL,INJURIES_FATAL,CRASH_MONTH,CRASH_DAY_OF_WEEK,CRASH_HOUR,LATITUDE,LONGITUDE
count,2.363271e+06,3512.000000,2.412776e+06,9.296760e+05,2.412776e+06,2.412752e+06,2.412776e+06,2.412776e+06,2.412776e+06,2.412776e+06,2.412776e+06,2.399324e+06,2.399324e+06
mean,5.402716e+05,0.165849,5.676385e+05,9.349354e+00,3.610831e+03,1.230165e+03,3.013156e-01,2.051164e-03,6.590377e+00,4.148680e+00,1.350763e+01,4.185222e+01,-8.766854e+01
std,3.077081e+05,0.098596,3.259501e+05,2.110810e+03,2.886118e+03,7.038798e+02,8.372557e-01,5.158671e-02,3.423331e+00,1.988100e+00,5.394394e+00,4.496020e-01,9.267125e-01
min,2.000000e+00,0.000000,2.000000e+00,0.000000e+00,0.000000e+00,1.110000e+02,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,-8.793476e+01
25%,2.752500e+05,0.120000,2.850108e+05,2.000000e+00,1.200000e+03,7.110000e+02,0.000000e+00,0.000000e+00,4.000000e+00,2.000000e+00,1.000000e+01,4.178215e+01,-8.772162e+01
50%,5.421820e+05,0.170000,5.697720e+05,2.000000e+00,3.136000e+03,1.134000e+03,0.000000e+00,0.000000e+00,7.000000e+00,4.000000e+00,1.400000e+01,4.187489e+01,-8.767357e+01
75%,8.059645e+05,0.210000,8.496552e+05,4.000000e+00,5.518000e+03,1.814000e+03,0.000000e+00,0.000000e+00,1.000000e+01,6.000000e+00,1.700000e+01,4.192198e+01,-8.763256e+01
max,1.071291e+06,1.000000,1.129364e+06,1.191625e+06,4.511000e+05,6.100000e+03,2.100000e+01,4.000000e+00,1.200000e+01,7.000000e+00,2.300000e+01,4.202278e+01,0.000000e+00


In [28]:
mergeddf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2412776 entries, 0 to 2412775
Data columns (total 38 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   CRASH_RECORD_ID        object 
 1   VEHICLE_ID             float64
 2   CRASH_DATE_x           object 
 3   INJURY_CLASSIFICATION  object 
 4   AIRBAG_DEPLOYED        object 
 5   DRIVER_ACTION          object 
 6   PEDPEDAL_ACTION        object 
 7   BAC_RESULT VALUE       float64
 8   CELL_PHONE_USE         object 
 9   CRASH_UNIT_ID          int64  
 10  CRASH_DATE_y           object 
 11  MAKE                   object 
 12  MODEL                  object 
 13  VEHICLE_TYPE           object 
 14  VEHICLE_DEFECT         object 
 15  FIRST_CONTACT_POINT    object 
 16  FIRE_I                 object 
 17  CRASH_DATE             object 
 18  WEATHER_CONDITION      object 
 19  LIGHTING_CONDITION     object 
 20  LANE_CNT               float64
 21  ROADWAY_SURFACE_COND   object 
 22  ROAD_DEFECT       

In [29]:
mergeddf['INJURIES_FATAL'].value_counts()

0.0    2408380
1.0       3991
2.0        297
3.0         68
4.0         40
Name: INJURIES_FATAL, dtype: int64

In [30]:
fataldf = mergeddf[mergeddf["INJURIES_FATAL"]>0]

In [31]:
fataldf

,CRASH_RECORD_ID,VEHICLE_ID,CRASH_DATE_x,INJURY_CLASSIFICATION,AIRBAG_DEPLOYED,DRIVER_ACTION,PEDPEDAL_ACTION,BAC_RESULT VALUE,CELL_PHONE_USE,CRASH_UNIT_ID,...,INJURIES_TOTAL,MOST_SEVERE_INJURY,INJURIES_FATAL,CRASH_MONTH,CRASH_DAY_OF_WEEK,TRAFFICWAY_TYPE,CRASH_HOUR,LATITUDE,LONGITUDE,LOCATION
1591,362c12430744ace0e1bd90e95689790166481ef62c096f...,795768.0,02/08/2020 09:47:00 PM,NONINCAPACITATING INJURY,"DEPLOYED, FRONT",TOO FAST FOR CONDITIONS,NaN,NaN,NaN,838562,...,7.0,FATAL,1.0,2,7,FOUR WAY,21,41.877436,-87.706134,POINT (-87.706134300119 41.877436205091)
1592,362c12430744ace0e1bd90e95689790166481ef62c096f...,795768.0,02/08/2020 09:47:00 PM,NONINCAPACITATING INJURY,"DEPLOYED, FRONT",TOO FAST FOR CONDITIONS,NaN,NaN,NaN,838563,...,7.0,FATAL,1.0,2,7,FOUR WAY,21,41.877436,-87.706134,POINT (-87.706134300119 41.877436205091)
1593,362c12430744ace0e1bd90e95689790166481ef62c096f...,795768.0,02/08/2020 09:47:00 PM,NONINCAPACITATING INJURY,"DEPLOYED, FRONT",TOO FAST FOR CONDITIONS,NaN,NaN,NaN,838564,...,7.0,FATAL,1.0,2,7,FOUR WAY,21,41.877436,-87.706134,POINT (-87.706134300119 41.877436205091)
1594,362c12430744ace0e1bd90e95689790166481ef62c096f...,795783.0,02/08/2020 09:47:00 PM,FATAL,"DEPLOYED, COMBINATION",NONE,NaN,NaN,NaN,838562,...,7.0,FATAL,1.0,2,7,FOUR WAY,21,41.877436,-87.706134,POINT (-87.706134300119 41.877436205091)
1595,362c12430744ace0e1bd90e95689790166481ef62c096f...,795783.0,02/08/2020 09:47:00 PM,FATAL,"DEPLOYED, COMBINATION",NONE,NaN,NaN,NaN,838563,...,7.0,FATAL,1.0,2,7,FOUR WAY,21,41.877436,-87.706134,POINT (-87.706134300119 41.877436205091)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2408139,96ba417b277d724284d2742aeedcbc399f23c29be9709e...,1069836.0,06/27/2021 07:26:00 PM,FATAL,DEPLOYMENT UNKNOWN,UNKNOWN,NaN,NaN,NaN,1127880,...,2.0,FATAL,1.0,6,1,ONE-WAY,19,41.871624,-87.718280,POINT (-87.718280084921 41.871623759129)
2408140,96ba417b277d724284d2742aeedcbc399f23c29be9709e...,NaN,06/27/2021 07:26:00 PM,"REPORTED, NOT EVIDENT",NaN,UNKNOWN,UNKNOWN/NA,NaN,NaN,1127843,...,2.0,FATAL,1.0,6,1,ONE-WAY,19,41.871624,-87.718280,POINT (-87.718280084921 41.871623759129)
2408141,96ba417b277d724284d2742aeedcbc399f23c29be9709e...,NaN,06/27/2021 07:26:00 PM,"REPORTED, NOT EVIDENT",NaN,UNKNOWN,UNKNOWN/NA,NaN,NaN,1127844,...,2.0,FATAL,1.0,6,1,ONE-WAY,19,41.871624,-87.718280,POINT (-87.718280084921 41.871623759129)
2408142,96ba417b277d724284d2742aeedcbc399f23c29be9709e...,NaN,06/27/2021 07:26:00 PM,"REPORTED, NOT EVIDENT",NaN,UNKNOWN,UNKNOWN/NA,NaN,NaN,1127845,...,2.0,FATAL,1.0,6,1,ONE-WAY,19,41.871624,-87.718280,POINT (-87.718280084921 41.871623759129)


In [32]:
fataldf.describe()

,VEHICLE_ID,BAC_RESULT VALUE,CRASH_UNIT_ID,LANE_CNT,STREET_NO,BEAT_OF_OCCURRENCE,INJURIES_TOTAL,INJURIES_FATAL,CRASH_MONTH,CRASH_DAY_OF_WEEK,CRASH_HOUR,LATITUDE,LONGITUDE
count,3.749000e+03,50.00000,4.396000e+03,1297.000000,4396.000000,4396.000000,4396.000000,4396.000000,4396.000000,4396.000000,4396.000000,4387.000000,4387.000000
mean,7.193862e+05,0.07240,7.531326e+05,3.025443,3643.397862,1063.680619,2.992721,1.125796,6.543904,4.137625,11.912648,41.830237,-87.673191
std,3.002430e+05,0.07981,3.147290e+05,5.534555,3063.284019,594.239503,2.195857,0.442208,3.064959,1.894591,8.127769,0.078718,0.054669
min,9.250000e+03,0.00000,9.672000e+03,0.000000,1.000000,111.000000,1.000000,1.000000,1.000000,1.000000,0.000000,41.647709,-87.933977
25%,4.458340e+05,0.00000,4.654368e+05,2.000000,800.000000,632.000000,1.000000,1.000000,5.000000,3.000000,3.000000,41.765291,-87.711428
50%,8.356370e+05,0.07000,8.724980e+05,2.000000,3130.000000,1012.000000,2.000000,1.000000,6.000000,5.000000,13.000000,41.846954,-87.678600
75%,9.701990e+05,0.11750,1.017374e+06,4.000000,5721.000000,1232.000000,5.000000,1.000000,9.000000,5.000000,20.000000,41.880986,-87.636723
max,1.069836e+06,0.30000,1.127880e+06,99.000000,13615.000000,2534.000000,10.000000,4.000000,12.000000,7.000000,23.000000,42.015842,-87.527618
